<a href="https://colab.research.google.com/github/mkornreich/MDtoGDocs/blob/main/Convert_Markdown_to_Google_Docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting Google Docs to Markdown

The example below shows how to convert Google Docs to Markdown. Once executed, observe the new markdown file (`video_notes.md`) is visible in https://drive.google.com/ and the new Google Docs file (`Video Notes`) is visible in https://drive.google.com/

In [2]:
# Mounting the Google Drive directory
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [21]:
# Authenticating with Google drive credentials
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from googleapiclient.discovery import build
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
creds = GoogleCredentials.get_application_default()
gauth.credentials = creds
drive = GoogleDrive(gauth)

service = build('drive', 'v3', credentials=creds)

In [9]:
MARKDOWN = '''
# Product Team Sync - May 15, 2023


## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)


## Agenda


### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration


### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues


### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates


## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal


## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders


## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets


---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
'''

In [40]:
# Create & upload a markdown file.
def upload_markdown(filename, contents):
  body = {
      'title': filename,
      'mimeType': 'text/markdown'

  }
  uploaded = drive.CreateFile(body)
  uploaded.SetContentString(contents)
  uploaded.Upload()
  if isinstance(uploaded,dict) and 'id' in uploaded:
    document_id = uploaded.get('id')
    print('Uploaded file with ID {}'.format(document_id))
    return document_id
  else:
    raise RuntimeError('Failed to upload markdwon')

In [41]:
document_id = upload_markdown('meeting_notes.md', MARKDOWN)

Uploaded file with ID 1Z9GzXCxPZ7FLcxNwQi8JRnz3aNsElVYl


After executing the cell above, a new file named 'meeting_notes.md' will appear in your [drive.google.com](https://drive.google.com/) file list.

In [38]:
# Converting the markdown to a Google Docs file
def convert_markdown_to_gdocs(document_id, new_title):
  mime_type = 'mimeType'
  gdocs_mime_type = 'application/vnd.google-apps.document'
  body = {
      'name': new_title,
      mime_type: gdocs_mime_type
  }

  response = service.files().copy(fileId=document_id, body=body).execute()

  if isinstance(response,dict) and mime_type in response and response[mime_type] == gdocs_mime_type:
    document_id = response.get('id')

    print(f'Created Google Doc: {new_title}')
    print(f'Document ID: {document_id}')
    print(f'Link: https://docs.google.com/document/d/{document_id}/edit')

    return "SUCCESS"
  else:
    raise RuntimeError(f'Could not create Google Doc from document_id {document_id}')

In [39]:
convert_markdown_to_gdocs(document_id, "Meeting Notes")

Created Google Doc: Meeting Notes
Document ID: 170k_q-aFyoEkSPTouwpppSdda01OUDOTPAW_Cc940gA
Link: https://docs.google.com/document/d/170k_q-aFyoEkSPTouwpppSdda01OUDOTPAW_Cc940gA/edit


'SUCCESS'

After executing the cell above, a new formatted Google Docs file named 'Meeting Notes' will appear in your [drive.google.com](https://drive.google.com/) file list.